# Lambda Codes

## Flight Data - Aerodatabox API

In [ ]:
# import api keys
from api_keys import *

In [2]:
def lambda_handler(event, context):
    
    # flights api
    airport_icoa = "EDDB"
    to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
    from_local_time = (datetime.now() + timedelta(hours=10)).strftime('%Y-%m-%dT%H:00')
    flight_api_key = "f6739f3056msh64ce9464977f614p147088jsn771af262d348"
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
    headers = {
        'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
        'x-rapidapi-key': flight_api_key
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    arrivals_berlin = response.json()['arrivals']
    arrivals_berlin = pd.DataFrame([get_flight_info(flight, airport_icoa) for flight in arrivals_berlin])

import json
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import requests
from datetime import datetime, timedelta
icoa = ['EDDB', 'EGLC', 'LEMD', 'LFPG']
berlin_icoa = icoa[0]
to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
from_local_time = (datetime.now() + timedelta(hours=11)).strftime('%Y-%m-%dT%H:00')
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{berlin_icoa}/{to_local_time}/{from_local_time}"
import requests
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }
berlin_flights= requests.request("GET", url, headers=headers, params=querystring)
from IPython.display import JSON
berlin_flights.json()
response = requests.request("GET", url, headers=headers, params=querystring)     
arrivals_berlin = response.json()['arrivals']     
arrivals_berlin = pd.DataFrame([get_flight_info(flight, icoa[0]) for flight in arrivals_be
#For London
london_icoa = icoa[1]
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{london_icoa}/{to_local_time}/{from_local_time}"
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
london_flights = requests.request("GET", url, headers=headers, params=querystring)
arrivals_london = london_flights.json()['arrivals']
arrivals_london = pd.DataFrame([get_flight_info(flight, icoa[1]) for flight in arrivals_london])
#For Madrid
madrid_icoa = icoa[2]
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{madrid_icoa}/{to_local_time}/{from_local_time}"
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
madrid_flights= requests.request("GET", url, headers=headers, params=querystring)
arrivals_madrid = madrid_flights.json()['arrivals']
arrivals_madrid = pd.DataFrame([get_flight_info(flight, icoa[2]) for flight in arrivals_madrid])
paris_icoa = icoa[3]
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{paris_icoa}/{to_local_time}/{from_local_time}"
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
paris_flights= requests.request("GET", url, headers=headers, params=querystring)
arrivals_paris = paris_flights.json()['arrivals']
arrivals_paris = pd.DataFrame([get_flight_info(flight, icoa[3]) for flight in arrivals_paris])
#Joining all arrivals in one Dataframe
(
 arrivals_data = 
 pd.concat([arrivals_berlin, 
            arrivals_london, 
            arrivals_madrid, 
            arrivals_paris], axis=0 )
)
arrivals_data.reset_index(drop = True, inplace=True)
# inserting data in our RDS
    schema="gans_pipe"
    host="gans.czzxdshmvaqk.eu-central-1.rds.amazonaws.com"
    user="admin" 
    password="MySQL_Password"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
 # flight arrivals data     
(arrivals_data         
         .replace({np.nan},'unknown')              
         .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))         
         .to_sql('arrivals', if_exists='append', con=con, index=False))

SyntaxError: invalid syntax (Temp/ipykernel_11556/781966928.py, line 26)

## Weather data - OWM API

In [ ]:
city_name = ['Berlin', 'London', 'Madrid', 'Paris']
country_name = ['DE', 'GB', 'ES', 'FR']
response_berlin = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[0]},{country_name[0]}&appid={OWM_key}&units=metric&lang=en')
from IPython.display import JSON
response_berlin.json()
#For Berlin
forecast_api = response_berlin.json()['list']
weather_info = []
# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[0] + ',' + country_name[0] #Check it
    weather_info.append(weather_hour)    
    
weather_data_berlin = pd.DataFrame(weather_info)
#For London
response_london = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[1]},{country_name[1]}&appid={OWM_key}&units=metric&lang=en')
from IPython.display import JSON
response_london.json()
#For London
forecast_api = response_london.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day
weather_info = []
# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[1] + ',' + country_name[1] 
    weather_info.append(weather_hour)    
    
weather_data_london = pd.DataFrame(weather_info)
For Madrid
response_madrid = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[2]},{country_name[2]}&appid={OWM_key}&units=metric&lang=en')
from IPython.display import JSON
response_madrid.json()
#For Madrid
forecast_api = response_madrid.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day
weather_info = []
# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[2] + ',' + country_name[2] 
    weather_info.append(weather_hour)    
    
weather_data_madrid = pd.DataFrame(weather_info)
#For Paris
response_paris = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[3]},{country_name[3]}&appid={OWM_key}&units=metric&lang=en')
from IPython.display import JSON
response_paris.json()
#For Paris
forecast_api = response_paris.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day
weather_info = []
# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[3] + ',' + country_name[3] #Check it
    weather_info.append(weather_hour)    
    
weather_data_paris = pd.DataFrame(weather_info)
#Joining all Dataframes in a single one
(
    weather_data = 
    pd.concat([weather_data_berlin, 
               weather_data_london, 
               weather_data_madrid, 
               weather_data_paris], 
              axis=0 )
)
# inserting weather data in our RDS
    schema="gans_pipe"
    host="gans.czzxdshmvaqk.eu-central-1.rds.amazonaws.com"
    user="admin" 
    password="MySQL_Password"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    # weather data
    (weather_data
        .assign(datetime = lambda x: pd.to_datetime(x['datetime']))
        .to_sql('weather', if_exists='append', con=con, index=False))